In [16]:
import numpy as np
import scipy.io as scio
import time
import onnxruntime
import adi

In [17]:
def GenerateSymbolMat(symbol_batch):
    _batch_size = symbol_batch.shape[0]
    _signal_dimension = symbol_batch.shape[1]
    _symbol_length = symbol_batch.shape[2]

    symbol_batch_real = np.real(symbol_batch)
    symbol_batch_imag = np.imag(symbol_batch)
    symbol_batch_mat = np.concatenate((symbol_batch_real,symbol_batch_imag), axis = 1).astype('float32')
    
    return symbol_batch_mat, _batch_size, _signal_dimension


# ZigBee
SymbolFilePath = './BeaconPacket.mat'
SymbolMat = 'BeaconPacket'

Symbol_file = scio.loadmat(SymbolFilePath)
Symbol_batch = Symbol_file[SymbolMat]
Symbol_batch = np.expand_dims(Symbol_batch,axis=0)


Beacon_Symbols, batch_size, signal_dimension = GenerateSymbolMat(Symbol_batch)
print(Beacon_Symbols.shape)
print(batch_size)
print(signal_dimension)

(1, 128, 28)
1
64


In [18]:
# Execution Provider of ONNX backend
EP_list = ['CPUExecutionProvider', 'CUDAExecutionProvider']
EPIdx = 1
print("Execution provider is", EP_list[EPIdx])
WiFi_session = onnxruntime.InferenceSession("WiFiMod.onnx",providers=[EP_list[EPIdx]])

Execution provider is CUDAExecutionProvider


In [19]:
ONNX_Input = {WiFi_session.get_inputs()[0].name: Beacon_Symbols}
ONNX_Output = WiFi_session.run(None, ONNX_Input)
ONNX_Output_array = ONNX_Output[0]

In [20]:
# Post processing for complex signals
realSig = ONNX_Output_array[:,:,0]
imagSig = ONNX_Output_array[:,:,1]
cplxSig = realSig + 1j*imagSig
print(cplxSig.shape)
# txWaveform = 6*cplxSig[0,:] * 2**14 # Range mapping
txWaveform = cplxSig[0,:] * 2**14 # Range mapping
print(txWaveform.shape)

(1, 2320)
(2320,)


In [21]:
sdr = adi.Pluto('ip:192.168.2.1') # or whatever your Pluto's IP is
centralFrequency = 2.462e9
samplingRate = 20e6
sdr.sample_rate = int(samplingRate) # sampling rate
sdr.tx_rf_bandwidth = int(samplingRate) # bandwidth
sdr.tx_lo = int(centralFrequency)
sdr.tx_hardwaregain_chan0 = 0 # Increase to increase tx power, valid range is -90 to 0 dB

In [22]:
numFrames = 100
sdr.tx_destroy_buffer()
sdr.tx_cyclic_buffer = False
for frameIdx in range(numFrames):
    sdr.tx(txWaveform)
    print("Send Beacon!")
    time.sleep(0.1)

Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!
Send Beacon!


KeyboardInterrupt: 

In [ ]:
# This part is for test. 
# NNwifiWaveform is waveform samples 
# generated by ONNX model running on laptop.
Waveform_file = scio.loadmat("./NNwifiWaveform.mat")
Waveform_mat = Waveform_file['NNwifiWaveform']
numFrames = 100
samples = 0.3*Waveform_mat[0,:]
samples *= 2**14
print(samples[:10])
print(txWaveform[:10])

In [ ]:
# sdr.tx_destroy_buffer()
# sdr.tx_cyclic_buffer = True
# sdr.tx(samples)
sdr.tx_destroy_buffer()
sdr.tx_cyclic_buffer = False
for frameIdx in range(numFrames):
    sdr.tx(samples)
    print("Send Beacon!")
    time.sleep(0.1)